In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autogen-agentchat~=0.2 -q
!pip install groq -q
!pip install pyautogen -q
!pip install ortools -q

## Autogen

In [3]:
import os
api_key='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'
# api_key='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'
config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama-3.3-70b-versatile",
        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": api_key,
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
        "temperature": 0
    }
]

In [4]:
from pathlib import Path
from typing import Tuple, List, Annotated, Literal, Optional
from autogen import AssistantAgent, UserProxyAgent, GroupChat, ConversableAgent, GroupChatManager
from autogen.coding import LocalCommandLineCodeExecutor

# Setting up the code executor
workdir = Path("/content/drive/MyDrive/FYP/coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Setting up the agents
human_input = UserProxyAgent(
    name="human_input",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg= lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": code_executor},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and "FINISH" in x.get("content", ""),
)

system_message = """You are a code agent of a AI Media production planning company.
Solve tasks using your coding skills. Prioritize using the provided functions.
In the following cases, suggest python code (in a python coding block) for the user to execute.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest.
The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: You MUST provide code. Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""

system_message = """You are a helpful AI assistant who writes code and the user executes it.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) for the user to execute.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: You can ONLY suggest code in Python, not other coding languages. Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""

code_agent = ConversableAgent(
    name="Code_Agent",
    system_message=system_message,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
)
system_message = """You are a instructor of a AI Media production planning company.
You specializes in logical reasoning, while your colleauge specializes in turning your logic to code.
Your job is to provide instruction with your language skills to your colleague for the next step.
Explain your thoughts then provide the suitable action to the code agent.
Follow the format of Reasoning and Acting. Show the reason in Reasons: and Action in Action:
IMPORTANT: You are just an instructor, your colleague will write the code. You MUST not write any code or scripts."""

instructor_agent = ConversableAgent(
    name="Instructor",
    system_message=system_message,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
)
group_chat = GroupChat(
    agents=[instructor_agent, code_agent, code_executor_agent],
    messages=[],
    max_round=19,
    send_introductions=True,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": config_list},
    system_message="You manage"
)

code_executor_agent.description = 'A computer terminal that performs no other action than running Python scripts provided by Code_Agent.'
code_agent.description = 'Responsible for writing and executing approved plans in Python scripts. The code generated is wrapped in a code block that no other user can modify it.'
instructor_agent.description = 'Responsible for providing instruction to Code_Agent.'

## Autogen tool

In [5]:
from drive.MyDrive.FYP.scheduler import *
import pandas as pd
from datetime import time, timedelta
import pytz

@code_executor_agent.register_for_execution()
@code_agent.register_for_llm(description="Generate a dynamic schedule with updated members or tasks. The schedule will be saved to the working directory")
def dynamic_schedulling(
    schedule_type: Annotated[Literal['member','task'],'Type of dynamic schedulling.'],
    schedule_file: Annotated[str, 'Current Schedule csv file path.'],
    members: Optional[Annotated[List[Member], 'List of members to be updated.']]=None,
    tasks: Optional[Annotated[List[Task], 'List of tasks to be selected.']]=None
) -> int:
  from drive.MyDrive.FYP.scheduler import Optimizer, Member, Task
  import pandas as pd
  from datetime import time, timedelta
  import pytz

  if schedule_type == 'member':
    assert members is not None, "Member list cannot be empty."
    assert tasks is None, "Task list cannot be filled."
  else:
    assert tasks is not None, "Task list cannot be empty."
    assert members is None, "Member list cannot be filled."

  all_tasks, all_members = get_members_and_tasks()

  # Get current time
  now_utc = datetime.now(pytz.utc)
  china_tz = pytz.timezone('Asia/Shanghai')
  now_china = now_utc.astimezone(china_tz).time()
  now_china = time(now_china.hour, now_china.minute)


  if schedule_type == 'member':
    # Replace members in all_members with matching IDs from members list
    for new_member in members:
        for i, existing_member in enumerate(all_members):
            if existing_member.id == new_member.id:
                all_members[i] = new_member
  else:
    for new_task in tasks:
      for i, existing_task in enumerate(all_tasks):
          if existing_task.id == new_task.id:
              all_tasks[i] = new_task

  df = pd.read_csv(schedule_file)
  df_filtered = filter_schedule(schedule_file_path=schedule_file, start_time=(now_china,'le'), output_type='dataframe')
  df_filtered['MemberID'] = df_filtered['MemberID'].astype(str)
  task_times = df_filtered.groupby('Task').agg({'Start': 'min', 'End': 'max', 'MemberID': lambda x: ','.join(x)})

  # Ensure the generated schedule does not change the finished / ongoing tasks time
  for index in task_times.index:
    task_id = int(index.split()[1])
    for task in all_tasks:
      if task.id == task_id:
        start_time = task_times.loc[index, 'Start']
        dt = datetime.combine(datetime.min, start_time)
        end_time = dt + timedelta(minutes=task.estimated_duration)

        task.time_of_day = [(start_time, end_time)]
        task.members = list(map(int,task_times.loc[index, 'MemberID'].split(',')))
        task.roles = []
      else:
        if f'Task {str(task.id)}' not in list(df_filtered['Task'].unique()):
          for i, timeslot in enumerate(task.time_of_day):
            start_time = timeslot[0]
            end_time = timeslot[1]
            if timeslot[0] < now_china:
              task.time_of_day[i] = (now_china, end_time)

  model = Optimizer(all_tasks, all_members)
  model.schedule_tasks(timeout=60)
  print('The optimized cost is: ', model.optimized_cost)
  model.export_schedule('/content/drive/MyDrive/FYP/coding')
  return model.optimized_cost


@code_executor_agent.register_for_execution()
@code_agent.register_for_llm(description="Get all members and tasks, including both completed and incomplete tasks.")
def get_members_and_tasks() -> Tuple[List[Task], List[Member]]:

  from openpyxl import load_workbook
  import pandas as pd
  from datetime import time
  from collections import defaultdict
  from drive.MyDrive.FYP.scheduler import Member, Task, ROLE_NAME

  workbook = load_workbook(filename='/content/drive/MyDrive/FYP/Test1_withMemberTaskExcel/Member&Task_Info.xlsx', data_only=True)
  sheet = workbook['Members']
  sheet2 = workbook['Tasks']

  member_names = []
  roles = []
  member_hr_rates = []
  member_ot_rates = []

  tasks = []
  members = []

  for cell in sheet[3][1:]:
      if cell.value:
          member_names.append(cell.value)

          col_index = cell.col_idx
          roles.append(sheet.cell(row=4, column=col_index).value)
          member_hr_rates.append(sheet.cell(row=5, column=col_index).value)
          member_ot_rates.append(sheet.cell(row=6, column=col_index).value)

  member_roles = [key for string in roles for key, value in ROLE_NAME.items() if value == string]

  def format_time(row_index):
      hours = row_index // 4
      minutes = (row_index % 4) * 15
      if hours == 24 and minutes == 0:
          return 23, 59
      else:
          return hours, minutes

  def process_unavailability(file_path, sheet_name="Members_availability"):
      df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
      times = df.iloc[:, 0]
      results = {}
      for col_idx in range(1, df.shape[1]):
          availability = df.iloc[:, col_idx].values
          intervals = []
          start = None
          for i, available in enumerate(availability):
              if available == 0 and start is None:
                  start = i
              elif available == 1 and start is not None:
                  intervals.append((format_time(start-1), format_time(i-1)))
                  start = None
          if start is not None:
              intervals.append((format_time(start-1), format_time(len(availability)-1)))
          results[col_idx-1] = intervals
      return results

  file_path = '/content/drive/MyDrive/FYP/Test1_withMemberTaskExcel/Member&Task_Info.xlsx'
  sheet_name = "Members_availability"
  unavailability_results = process_unavailability(file_path, sheet_name)
  member_intervals = defaultdict(list)

  for person, intervals in unavailability_results.items():
      member_intervals[person].extend(intervals)

  for i, name in enumerate(member_names):
      converted_intervals = [
          (time(interval[0][0], interval[0][1]), time(interval[1][0], interval[1][1])) for interval in member_intervals[i]]
      members.append(Member(id=i, name=name, rate=member_hr_rates[i], ot=member_ot_rates[i], role=member_roles[i], blocked_timeslots=converted_intervals))

  task_names = defaultdict(str)
  task_locations = defaultdict(list)
  task_durations = defaultdict(str)
  task_dependencies = defaultdict(list)
  # task_timeofday = {}
  task_actors = defaultdict(list)
  task_roles = defaultdict(list)

  for i, row in enumerate(range(3, sheet2.max_row+1)):
      cell_value = sheet2.cell(row=row, column=3).value
      if cell_value:
          task_names[i] = cell_value
          task_locations[i] = eval(sheet2.cell(row=row, column = 9).value)
          task_durations[i] = sheet2.cell(row=row, column = 10).value
          task_dependencies[i] = eval(sheet2.cell(row=row, column = 12).value)
          task_actors[i] = eval(sheet2.cell(row=row, column = 16).value)
          task_roles[i] = eval(sheet2.cell(row=row, column = 25).value)

  for i, name in task_names.items():
      task_location = task_locations[i]
      task_duration = task_durations[i]
      task_actor = task_actors[i]
      task_role = task_roles[i]
      task_dependency = task_dependencies[i]
      tasks.append(Task(id=i, estimated_duration=task_duration, location=task_location, description=name, members=task_actor, roles=task_role, dependencies=task_dependency))

  return tasks, members

# @code_executor_agent.register_for_execution()
# @code_agent.register_for_llm(description="Generate an optimized schedule and returns a cost. The schedule csv and timetable will be saved in the working directory.")
def generate_optimized_schedule(
    tasks: Annotated[List[Task], 'List of tasks yet to be completed.'],
    members: Annotated[List[Member], 'List of members that can be chosen.'],
    num_workers = 8) -> int:

  from drive.MyDrive.FYP.scheduler import Member, Task, ROLE_NAME

  model = Optimizer(tasks, members)
  model.schedule_tasks(timeout=60, num_workers = num_workers)
  print('The optimized cost is: ', model.optimized_cost)
  model.export_schedule('/content/drive/MyDrive/FYP/Output')

  return model.optimized_cost


@code_executor_agent.register_for_execution()
@code_agent.register_for_llm(description="Filter the current schedule by Member name, Task ID, Start time, Expected End time, or Duration. Returns the filted schedule in a markdown table.")
def filter_schedule(
    members: Optional[Annotated[List[str], 'List of members to be selected.']]=None,
    tasks: Optional[Annotated[List[int], 'List of tasks ids to be selected.']]=None,
    start_time: Optional[Annotated[Tuple[time, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with start time different operator.']]=None,
    end_time: Optional[Annotated[Tuple[time, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with end time with different operator.']]=None,
    duration: Optional[Annotated[Tuple[int, Literal['ge', 'le', 'gt', 'lt', 'eq']], 'Filter tasks with a duration that fits the duration requirement.']]=None,
    output_type: Optional[Literal['markdown','dataframe']]='markdown',
    schedule_file_path: Optional[str] = '/content/drive/MyDrive/FYP/coding/schedule_solution_1.csv') -> str:

    import pandas as pd

    df = pd.read_csv(schedule_file_path)
    df['Start'] = pd.to_datetime(df['Start']).dt.time
    df['End'] = pd.to_datetime(df['End']).dt.time

    if members:
        df = df[df['Member'].isin(members)]
    if tasks:
        df = df[df['Task'].isin(str(tasks))]
    if start_time:
      if start_time[1] =='ge':
        df = df[df['Start'] >= start_time[0]]
      elif start_time[1] == 'le':
        df = df[df['Start'] <= start_time[0]]
      elif start_time[1] == 'gt':
        df = df[df['Start'] > start_time[0]]
      elif start_time[1] == 'lt':
        df = df[df['Start'] < start_time[0]]
      elif start_time[1] == 'eq':
        df = df[df['Start'] == start_time[0]]
    if end_time:
      if end_time[1] == 'ge':
        df = df[df['End'] >= end_time[0]]
      elif end_time[1] == 'le':
        df = df[df['End'] <= end_time[0]]
      elif end_time[1] == 'gt':
        df = df[df['End'] > end_time[0]]
      elif end_time[1] == 'lt':
        df = df[df['End'] < end_time[0]]
      elif end_time[1] == 'eq':
        df = df[df['End'] == end_time[0]]

    if duration:
      if duration[1] == 'ge':
        df = df[df['Duration'] >= duration[0]]
      elif duration[1] == 'le':
        df = df[df['Duration'] <= duration[0]]
      elif duration[1] == 'gt':
        df = df[df['Duration'] > duration[0]]
      elif duration[1] == 'lt':
        df = df[df['Duration'] < duration[0]]
      elif duration[1] == 'eq':
        df = df[df['Duration'] == duration[0]]

    return df.to_markdown(index=False) if output_type == 'markdown' else df


ImportError: cannot import name 'MemberBase' from 'drive.MyDrive.FYP.scheduler.scheduler_classes' (/content/drive/MyDrive/FYP/scheduler/scheduler_classes.py)

## Schedulling test

In [ ]:
tasks, members = get_members_and_tasks()
i = 0
for num in [8,16,32,64]:
  while i < 5:
      generate_optimized_schedule(tasks, members, num_workers=num)
      i += 1
  i = 0

2
The optimized cost is:  30899.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30885.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30474.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30785.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30710.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30721.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  31746.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  31105.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30899.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  30721.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  31441.0
Solution 1 saved to /content/drive/MyDrive/FYP/Output
2
The optimized cost is:  31746.

## Dynamic Schedulling test

In [ ]:
dynamic_schedulling(
    schedule_type='member',
    schedule_file='/content/drive/MyDrive/FYP/Output/schedule_solution_1.csv',
    members=[Member(id=15, name='Camera1', rate=50, ot=50, role=4, blocked_timeslots=[(time(0, 0), time(9, 0)), (time(16, 0), time(23, 59))])])

In [ ]:
dynamic_schedulling(
    schedule_type='task',
    schedule_file='/content/drive/MyDrive/FYP/Output/schedule_solution_1.csv',
    tasks=[Task(id=17, estimated_duration=75, location=['The Galleria', 'Studio'], description='RayPhotoshoot', dependencies=[20], time_of_day=[(time(0, 0), time(23, 59))], members=[3], roles=[(1, 2), (4, 2)])])

## Agent test

In [ ]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
chat_result = code_executor_agent.initiate_chat(
    code_agent,
    message="The time now is 15:50. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. What are the tasks that should be completed by now? And who are the members that are on duty now?",
)
# chat_result = code_executor_agent.initiate_chat(
#     code_agent,
#     message="The time now is 15:58. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. All tasks apart from task Jacob's shot are finished by schedule. Jacob's shot is expected to finish by 16:00, but we expect a 30 mins delay. What are the updated costs and schedule? You are required to prioritize using provided functions over using code.",
# )
# task = 'All tasks apart from task 20 are finished by schedule. Task 20 is expected to finish by now, but we expect a 30 mins delay. Update the schedule for the rest of the day.'"You work for a media production company. Please solve the following task: {task}\nThe time now is 12:30. The timetable can be achieved by calling the provided functions. Note that start and end times are expected times."
# result = code_executor_agent.initiate_chat(
#     code_agent,
#     message=f"The time now is 15:50. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. What are the tasks that should be completed by now? And who are the members that are on duty now?")

code_executor_agent (to Code_Agent):

The time now is 15:50. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. What are the tasks that should be completed by now? And who are the members that are on duty now?

--------------------------------------------------------------------------------
Code_Agent (to code_executor_agent):

```python
import pandas as pd
from datetime import datetime

# Load the schedule
schedule = pd.read_csv('schedule_solution_1.csv')

# Convert start and end times to datetime format
schedule['start_time'] = pd.to_datetime(schedule['start_time'])
schedule['end_time'] = pd.to_datetime(schedule['end_time'])

# Get the current time
current_time = datetime.strptime('15:50', '%H:%M')

# Find tasks that should be completed by now
completed_tasks = schedule[schedule['end_time'] <= current_time]

# Find members who are on duty now
on_duty_

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jqvb88vkfx8922yyzg9yt0nk` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 101619, Requested 5648. Please try again in 1h44m39.505s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
# chat_result = code_executor_agent.initiate_chat(
#     code_agent,
#     message="The time now is 15:50. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. What are the tasks that should be completed by now? And who are the members that are on duty now?",
# )
chat_result = code_executor_agent.initiate_chat(
    code_agent,
    message="The time now is 15:58. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. All tasks apart from task Jacob's shot are finished by schedule. Jacob's shot is expected to finish by 16:00, but we expect a 30 mins delay. What are the updated costs and schedule? You are required to prioritize using provided functions over using code.",
)
# task = 'All tasks apart from task 20 are finished by schedule. Task 20 is expected to finish by now, but we expect a 30 mins delay. Update the schedule for the rest of the day.'"You work for a media production company. Please solve the following task: {task}\nThe time now is 12:30. The timetable can be achieved by calling the provided functions. Note that start and end times are expected times."
# result = code_executor_agent.initiate_chat(
#     code_agent,
#     message=f"The time now is 15:50. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. What are the tasks that should be completed by now? And who are the members that are on duty now?")

code_executor_agent (to Code_Agent):

The time now is 15:58. The timetable is provided in the work directory in a csv file called schedule_solution_1.csv. It shows each member's responsible tasks. Note that start and end times are expected times. All tasks apart from task Jacob's shot are finished by schedule. Jacob's shot is expected to finish by 16:00, but we expect a 30 mins delay. What are the updated costs and schedule? You are required to prioritize using provided functions over using code.

--------------------------------------------------------------------------------
Code_Agent (to code_executor_agent):

***** Suggested tool call (call_vra9): dynamic_schedulling *****
Arguments: 
{"schedule_file": "schedule_solution_1.csv", "schedule_type": "task", "members": null, "tasks": null}
****************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION dynamic_schedulling...


/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model llama-3.3-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Code_Agent (to code_executor_agent):

***** Suggested tool call (call_3bzh): get_members_and_tasks *****
Arguments: 
{}
******************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_members_and_tasks...
Call ID: call_3bzh
Input arguments: {}
code_executor_agent (to Code_Agent):

***** Response from calling tool (call_3bzh) *****
([{'id': 0, 'estimated_duration': 30, 'location': ['Parking Lot Near Loading Bay'], 'description': 'JacobMakeup', 'dependencies': [], 'time_of_day': [(datetime.time(0, 0), datetime.time(23, 59))], 'members': [0], 'roles': [(2, 1)]}, {'id': 1, 'estimated_duration': 75, 'location': ['Parking Lot Near Loading Bay'], 'description': 'SetupLightingParkingLot', 'dependencies': [], 'time_of_day': [(datetime.time(0, 0), datetime.time(23, 59))], 'members': [], 'roles': [(1, 5), (4, 2)]}, {'id': 2, 'estimated_duration': 75, 'location': ['Parking Lot Near

/usr/local/lib/python3.11/dist-packages/autogen/oai/groq.py:303: UserWarning: Cost calculation not available for model llama-3.3-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Code_Agent (to code_executor_agent):

```python
# Import necessary libraries
import pandas as pd
from datetime import datetime, timedelta

# Load the schedule from the csv file
schedule = pd.read_csv('schedule_solution_1.csv')

# Get the current time
current_time = datetime.strptime('15:58', '%H:%M')

# Get the task 'Jacob's shot'
jacob_shot_task = schedule[schedule['Task'] == 'Jacob\'s shot']

# Calculate the new end time of 'Jacob's shot' with a 30 minutes delay
new_end_time = current_time + timedelta(minutes=30)

# Update the end time of 'Jacob's shot' in the schedule
schedule.loc[jacob_shot_task.index, 'End Time'] = new_end_time.strftime('%H:%M')

# Calculate the updated costs and schedule
updated_costs = schedule['Cost'].sum()
updated_schedule = schedule

# Print the updated costs and schedule
print('Updated Costs:', updated_costs)
print('Updated Schedule:')
print(updated_schedule)
```

--------------------------------------------------------------------------------

>>>>>>>> EXEC

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jjbpczkhfzs9zdnxrezywbp5` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 8220, Requested 3822. Please try again in 1m0.421s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}